In [1]:
!rm -rf dqm_repo
!feast init dqm_repo
%cd dqm_repo

/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(

Creating a new Feast repository in /Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo.

/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo


In [2]:
import pandas as pd

pd.read_parquet("data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-08-30 11:00:00+00:00,1005,0.452095,0.635198,696,2022-09-14 11:40:22.613
1,2022-08-30 12:00:00+00:00,1005,0.278842,0.491479,807,2022-09-14 11:40:22.613
2,2022-08-30 13:00:00+00:00,1005,0.199585,0.544063,44,2022-09-14 11:40:22.613
3,2022-08-30 14:00:00+00:00,1005,0.234893,0.304014,715,2022-09-14 11:40:22.613
4,2022-08-30 15:00:00+00:00,1005,0.462757,0.025234,425,2022-09-14 11:40:22.613
...,...,...,...,...,...,...
1802,2022-09-14 09:00:00+00:00,1001,0.034532,0.964205,902,2022-09-14 11:40:22.613
1803,2022-09-14 10:00:00+00:00,1001,0.045746,0.353958,86,2022-09-14 11:40:22.613
1804,2021-04-12 07:00:00+00:00,1001,0.238023,0.343046,1,2022-09-14 11:40:22.613
1805,2022-09-06 23:00:00+00:00,1003,0.155819,0.030764,263,2022-09-14 11:40:22.613


In [3]:
!feast apply

/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
Created entity driver
Created feature view driver_hourly_stats
Created feature service driver_activity

Created sqlite table dqm_repo_driver_hourly_stats



In [4]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage
from feast.dqm.profilers.ge_profiler import ge_profiler
from great_expectations.dataset import PandasDataset
from great_expectations.core.expectation_suite import ExpectationSuite

# The entity dataframe is the dataframe we want to enrich with feature values
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "label_driver_reported_satisfaction": [1, 5, 3], 
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
        "created": datetime.now(),
    }
)

store = FeatureStore(repo_path=".")

training_data_job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ], 
)

/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/great_expectations/marshmallow__shade/__init__.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  __version_info__ = tuple(LooseVersion(__version__).version)


In [5]:
training_data_job.to_df()

,driver_id,label_driver_reported_satisfaction,event_timestamp,created__,conv_rate,acc_rate,avg_daily_trips
0,1002,5,2022-09-14 11:04:58.587163+00:00,2022-09-14 11:40:22.613,0.825219,0.345714,569
1,1001,1,2022-09-14 11:29:58.587153+00:00,2022-09-14 11:40:22.613,0.045746,0.353958,86
2,1003,3,2022-09-14 10:27:58.587165+00:00,2022-09-14 11:40:22.613,0.531909,0.257000,982


In [6]:
reference_dataset = store.create_saved_dataset(
    from_=training_data_job,
    name="reference_dataset",
    storage=SavedDatasetFileStorage(path='data/driver_stats.parquet')
)

print(reference_dataset)

{
  "spec": {
    "name": "reference_dataset",
    "features": [
      "driver_hourly_stats:conv_rate",
      "driver_hourly_stats:acc_rate",
      "driver_hourly_stats:avg_daily_trips"
    ],
    "joinKeys": [
      "created",
      "driver_id",
      "label_driver_reported_satisfaction"
    ],
    "storage": {
      "fileStorage": {
        "fileFormat": {
          "parquetFormat": {}
        },
        "uri": "data/driver_stats.parquet"
      }
    }
  },
  "meta": {
    "createdTimestamp": "2022-09-14T17:41:13.968677Z",
    "minEventTimestamp": "2022-09-14T10:27:58.587165Z",
    "maxEventTimestamp": "2022-09-14T11:29:58.587153Z"
  }
}


/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1127: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [7]:
@ge_profiler
def user_features_profiler(dataset: PandasDataset) -> ExpectationSuite:
    print(dataset.columns)
    dataset.expect_column_to_exist("driver_id")
    dataset.expect_column_values_to_be_between("driver_hourly_stats__avg_daily_trips", 0, 1000)
    dataset.expect_column_values_to_be_between("driver_hourly_stats__conv_rate", 0, 1)
    dataset.expect_column_values_to_be_between("driver_hourly_stats__acc_rate", 0, 1)
    return dataset.get_expectation_suite()

ds = store.get_saved_dataset('reference_dataset')

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1180: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [8]:
validation_reference = ds.as_reference(name='user_features_profiler', profiler=user_features_profiler)
_ = training_data_job.to_df(validation_reference=validation_reference)

KeyError: "['created'] not in index"

In [ ]:
training_data_job.to_df()